In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193676 sha256=e62afa005f58d3e13698ff3e01a92ac0fa57aab342652235734db2f0ccd9a6d9
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [40]:
import numpy as np
import pandas as pd 
import math # 불러오기 수학과 관련된 함수
import re # 정규표현식 함수 , 판별 여부
from scipy.sparse import csr_matrix # 매트릭스 해줌
import matplotlib.pyplot as plt # 시각화 함수
import seaborn as sns # 시각화 함수
from surprise import Reader, Dataset, SVD, NormalPredictor, KNNBasic # 알고리즘이 들어감
from surprise import KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import BaselineOnly, SVDpp,NMF, SlopeOne, CoClustering # 분석툴
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import cross_validate, KFold, train_test_split # 교차 검증
import json
import mpmath
from scipy.special import logsumexp

from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ml_plus/data')

In [ ]:
from surprise.model_selection import GridSearchCV, cross_validate

In [ ]:
ratings = pd.read_csv('my_rating.csv')
movies = pd.read_csv('영화 & TV 프로그램 .csv', index_col= 0)

In [ ]:
ratings = ratings.drop(['contentReviewId','title','content','createdAt','updatedAt'],axis=1)

In [ ]:
ratings.head(2)

,contentReviewUserId,contentId,userRating
0,439,93,1.0
1,143,848,3.9


In [ ]:
movies = movies.drop(['content','imgUrl','createdYear','type'],axis=1)

In [ ]:
movies.head(2)

,contentId,title,genre,contentRating
0,631842,똑똑똑,"['Horror', 'Mystery', 'Thriller']",6.6
1,505642,블랙 팬서: 와칸다 포에버,"['Action', 'Adventure', 'Science Fiction']",7.4


In [ ]:
ratings.dtypes

contentReviewUserId      int64
contentId                int64
userRating             float64
dtype: object

In [ ]:
movies.dtypes

contentId         object
title             object
genre             object
contentRating    float64
dtype: object

In [ ]:
ratings['contentId'] = ratings['contentId'].astype('string')

In [ ]:
rating_movies = pd.merge(ratings, movies, on='contentId')
rating_movies

,contentReviewUserId,contentId,userRating,title,genre,contentRating
0,439,93,1.0,살인자의 해부,"['Crime', 'Drama', 'Mystery', 'Thriller']",7.8
1,439,93,1.0,Falcon Crest,"['Soap', 'Drama']",5.7
2,642,93,4.9,살인자의 해부,"['Crime', 'Drama', 'Mystery', 'Thriller']",7.8
3,642,93,4.9,Falcon Crest,"['Soap', 'Drama']",5.7
4,605,93,1.1,살인자의 해부,"['Crime', 'Drama', 'Mystery', 'Thriller']",7.8
...,...,...,...,...,...,...
1664,4,786,4.4,올모스트 페이머스,"['Drama', 'Music']",7.5
1665,3,345,1.0,아이즈 와이드 셧,"['Drama', 'Thriller', 'Mystery']",7.5
1666,3,345,1.0,My Little Pony,['Animation'],7.4
1667,9,657,1.1,007 위기일발,"['Action', 'Thriller', 'Adventure']",7.1


In [ ]:
ratings_matrix = rating_movies.pivot_table('userRating', index='contentReviewUserId', columns='title')
ratings_matrix.fillna(0, inplace=True)
ratings_matrix.head(2)

title,007 골드핑거,007 골든 아이,007 나를 사랑한 스파이,007 네버 다이,007 다이아몬드는 영원히,007 두번 산다,007 리빙 데이라이트,007 문레이커,007 뷰 투 어 킬,007 살인 면허,...,혹성탈출,화성 침공,환타지아,황금광 시대,황야의 무법자,황혼에서 새벽까지,후크,흐르는 강물처럼,히글리타운의 영웅들,히트
contentReviewUserId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,4.6,0.0,0.0,0.0,0.0,...,2.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.7,0.0,2.4,0.0,0.0,0.0,0.0,0.0,0.0,4.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7,0.0,0.0


In [ ]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(2)

contentReviewUserId,1,2,3,4,5,6,7,8,9,10,...,988,989,990,992,993,995,996,997,998,999
title,,,,,,,,,,,,,,,,,,,,,
007 골드핑거,0.0,2.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
007 골든 아이,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(2)

(730, 730)


title,007 골드핑거,007 골든 아이,007 나를 사랑한 스파이,007 네버 다이,007 다이아몬드는 영원히,007 두번 산다,007 리빙 데이라이트,007 문레이커,007 뷰 투 어 킬,007 살인 면허,...,혹성탈출,화성 침공,환타지아,황금광 시대,황야의 무법자,황혼에서 새벽까지,후크,흐르는 강물처럼,히글리타운의 영웅들,히트
title,,,,,,,,,,,,,,,,,,,,,
007 골드핑거,1.0,0.0,0.523016,0.0,0.0,0.0,0.0,0.0,0.0,0.402596,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.291372,0.0,0.0
007 골든 아이,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [ ]:
def find_sim_movie_item(df, title_name, top_n=10):
    title_movie_sim = df[[title_name]].drop(title_name, axis=0)

    return title_movie_sim.sort_values(title_name, ascending=False)[:top_n]

In [ ]:
find_sim_movie_item(item_sim_df, '후크')

title,후크
title,
All My Children,1.000000
Colditz,0.616602
어바웃 어 보이,0.616602
투모로우,0.616602
Tom and Jerry Tales,0.616602
데드 맨 워킹,0.616602
진주만,0.616602
폴터가이스트,0.616602
아웃사이더,0.593909
